In [1]:
import requests
import json
import pandas as pd
import numpy as np
import datetime as dt
from bs4 import BeautifulSoup as soup
import os

In [ ]:
csv='../Resources/APPT_PRD_Run.csv'
csv_history = '../Exports/PRD_full.csv'
PRD_full = pd.read_csv(csv_history)
prd_run_df= pd.read_csv(csv)
prd_run_df.head()

In [ ]:
prd_run_df.columns

In [4]:
# prd_run_df.dropna(subset=['APPT ID'], inplace=True)
# prd_run_df['APPT ID'] = prd_run_df['APPT ID'].astype(int)

In [ ]:
prd_run_df.dropna(subset=['APPT ID'], inplace=True)
prd_run_df.tail()

In [ ]:
po_count = len(prd_run_df['APPT ID'].unique())
po_count

In [7]:
# prd_run_df = prd_run_df[(prd_run_df['APPT PRD FLAG'] == 'Appt without PRD Run')]

In [ ]:
PRD_count = len(prd_run_df['APPT ID'].unique())
PRD_count

In [9]:
prd_run_df[["Date_Worked","Time_Worked"]] = prd_run_df["APPT OPEN TS"].str.split(expand=True)

In [10]:
prd_run_df['APPT OPEN TS'] = pd.to_datetime(prd_run_df['APPT OPEN TS'])

In [ ]:
prd_run_df.dtypes

In [ ]:
prd_run_df['Weekday']= prd_run_df['APPT OPEN TS'].dt.weekday
prd_run_df.head()

In [ ]:
shift_a1_start = '06:00:00'
shift_a1_end = '18:00:00'

shift_a2_start = '18:00:00'
shift_a2_new = '00:00:00'
shift_a2_mid = '23:59:59'
shift_a2_end = '06:00:00'

shift_b1_start = '06:00:00'
shift_b1_end = '16:00:00'

shift_b2_start = '16:00:00'
shift_b2_new = '00:00:00'
shift_b2_mid = '23:59:59'
shift_b2_end = '02:00:00'


prd_run_df['A1'] = ((prd_run_df['APPT OPEN TS'].dt.dayofweek >= 5) | (prd_run_df['APPT OPEN TS'].dt.dayofweek == 0)) & \
                 ((prd_run_df['APPT OPEN TS'].dt.time >= pd.to_datetime(shift_a1_start).time()) & \
                  (prd_run_df['APPT OPEN TS'].dt.time <= pd.to_datetime(shift_a1_end).time()))

prd_run_df['A2'] = (((prd_run_df['APPT OPEN TS'].dt.dayofweek >= 5) | (prd_run_df['APPT OPEN TS'].dt.dayofweek == 0)) & \
                 (prd_run_df['APPT OPEN TS'].dt.time >= pd.to_datetime(shift_a2_start).time()) & \
                  (prd_run_df['APPT OPEN TS'].dt.time <= pd.to_datetime(shift_a2_mid).time())) | \
                (((prd_run_df['APPT OPEN TS'].dt.dayofweek == 6) | (prd_run_df['APPT OPEN TS'].dt.dayofweek <= 1)) & \
                 (prd_run_df['APPT OPEN TS'].dt.time >= pd.to_datetime(shift_a2_new).time()) & \
                  (prd_run_df['APPT OPEN TS'].dt.time <= pd.to_datetime(shift_a2_end).time()))

prd_run_df['B1'] = (prd_run_df['APPT OPEN TS'].dt.dayofweek >= 1) & (prd_run_df['APPT OPEN TS'].dt.dayofweek <= 4) & \
                 ((prd_run_df['APPT OPEN TS'].dt.time >= pd.to_datetime(shift_b1_start).time()) & \
                  (prd_run_df['APPT OPEN TS'].dt.time <= pd.to_datetime(shift_b1_end).time()))

prd_run_df['B2'] = ((prd_run_df['APPT OPEN TS'].dt.dayofweek >= 1) & (prd_run_df['APPT OPEN TS'].dt.dayofweek <= 4) & \
                 ((prd_run_df['APPT OPEN TS'].dt.time >= pd.to_datetime(shift_b2_start).time()) & \
                  (prd_run_df['APPT OPEN TS'].dt.time <= pd.to_datetime(shift_b2_mid).time()))) | \
                 ((prd_run_df['APPT OPEN TS'].dt.dayofweek >= 2) & (prd_run_df['APPT OPEN TS'].dt.dayofweek <= 5) & \
                 ((prd_run_df['APPT OPEN TS'].dt.time >= pd.to_datetime(shift_b2_new).time()) & \
                  (prd_run_df['APPT OPEN TS'].dt.time <= pd.to_datetime(shift_b2_end).time())))

prd_run_df.head()

In [ ]:
prd_run_df['Shift']= np.where(prd_run_df['A1'], 'A1',
                                np.where(prd_run_df['A2'], 'A2',
                                np.where(prd_run_df['B1'], 'B1',
                                np.where(prd_run_df['B2'], 'B2',
                                'B3'))))

prd_run_df.head()

In [15]:
# prd_run_df['Week_Number'] = prd_run_df['APPT OPEN TS'].dt.isocalendar().week

In [16]:
PRD_full = pd.concat([PRD_full, prd_run_df], axis=0)
PRD_full.drop_duplicates(subset=['APPT ID'], keep='first', inplace=True)

In [17]:
PRD_full.to_csv('../Exports/PRD_full.csv', index=True)
prd_run_df.to_csv('../Exports/PRD_Shift.csv', index=True)

In [18]:
prd_run_total = prd_run_df

In [ ]:
total_open_key = prd_run_total.groupby('Shift')['APPT OPEN TS'].count()
total_open_key = total_open_key.reset_index(name='Total_Run')
total_open_key

In [ ]:
prd_run_total = prd_run_total[(prd_run_total['APPT PRD FLAG'] == 'Appt without PRD Run')]
prd_run_total.head()

In [21]:
# prd_run_total.to_csv('../Exports/Outside_PRD.csv', index=True)

In [ ]:
out_prd_key = prd_run_total.groupby('Shift')['APPT ID'].count()
out_prd_key = out_prd_key.reset_index(name='Out_PRD')
out_prd_key

In [ ]:
total_prd = pd.merge(total_open_key, out_prd_key, how='left', on=['Shift', 'Shift'])
total_prd['Out_PRD'] = total_prd['Out_PRD'].fillna(0)
total_prd['Out_PRD'] = total_prd['Out_PRD'].astype('int')
total_prd

In [24]:
total_prd.to_csv('../Exports/Total_PRD.csv', index=True)

In [ ]:
csvsun='../Resources/sunday.csv'
csvmon='../Resources/monday.csv'
csvtue='../Resources/tuesday.csv'
csvwed='../Resources/wednesday.csv'
# csvthu='../Resources/thursday.csv'
# csvfri='../Resources/friday.csv'
# csvsat='../Resources/saturday.csv'

csvplan='../Exports/plan_run_full.csv'

run_sun= pd.read_csv(csvsun)
run_mon= pd.read_csv(csvmon)
run_tue= pd.read_csv(csvtue)
run_wed= pd.read_csv(csvwed)
# run_thu= pd.read_csv(csvthu)
# run_fri= pd.read_csv(csvfri)
# run_sat= pd.read_csv(csvsat)

plan_run_full= pd.read_csv(csvplan)

run_sun['Plan_Run_Weekday'] = 'Sunday'
run_mon['Plan_Run_Weekday'] = 'Monday'
run_tue['Plan_Run_Weekday'] = 'Tuesday'
run_wed['Plan_Run_Weekday'] = 'Wednesday'
# run_thu['Plan_Run_Weekday'] = 'Thursday'
# run_fri['Plan_Run_Weekday'] = 'Friday'
# run_sat['Plan_Run_Weekday'] = 'Saturday'

run_sun.dropna(subset=['APPOINTMENT NUMBER\t'], inplace=True)
run_mon.dropna(subset=['APPOINTMENT NUMBER\t'], inplace=True)
run_tue.dropna(subset=['APPOINTMENT NUMBER\t'], inplace=True)
run_wed.dropna(subset=['APPOINTMENT NUMBER\t'], inplace=True)
# run_thu.dropna(subset=['APPOINTMENT NUMBER\t'], inplace=True)
# run_fri.dropna(subset=['APPOINTMENT NUMBER\t'], inplace=True)
# run_sat.dropna(subset=['APPOINTMENT NUMBER\t'], inplace=True)


# full_run = pd.concat([run_sun, run_mon, run_tue, run_wed, run_thu, run_fri, run_sat], axis=0)
full_run = pd.concat([run_sun, run_mon, run_tue, run_wed], axis=0)

full_run

In [26]:
full_run = full_run.rename(columns={'APPOINTMENT NUMBER\t': 'APPT ID'})
# full_run.head()

In [27]:
full_run.to_csv('../Exports/Weekly_Plan.csv', index=True)
prd_run_total.to_csv('../Exports/Outside_PRD.csv', index=True)

In [28]:
plan_run_full = pd.concat([plan_run_full, full_run], axis=0)
plan_run_full.drop_duplicates(subset=['APPT ID'], keep='first', inplace=True)

In [29]:
plan_run_full.to_csv('../Exports/plan_run_full.csv', index=True)

In [30]:
# full_count = full_run['NODE'].count()
# full_count

In [ ]:
run_analysis = pd.merge(prd_run_df,plan_run_full[['APPT ID', 'Plan_Run_Weekday']], on='APPT ID', how='left')
run_analysis

In [ ]:
count = run_analysis['TRAILER NUM'].isna().sum()
count

In [ ]:
run_analysis_notnull = run_analysis[run_analysis['TRAILER NUM'].notnull()]
run_analysis_notnull

In [ ]:
run_analysis_notnull = run_analysis_notnull.drop(run_analysis_notnull[run_analysis_notnull['TRAILER NUM'].str.startswith('L')].index)
run_analysis_notnull

In [ ]:
count = run_analysis_notnull['Plan_Run_Weekday'].isna().sum()
count

In [ ]:
Final_analysis = run_analysis_notnull[run_analysis_notnull['Plan_Run_Weekday'].notnull()]
Final_analysis

In [37]:
# run_analysis_null = run_analysis_notnull[run_analysis_notnull['Plan_Run_Weekday'].isna()]
# run_analysis_null

In [ ]:
def custom_day_of_week(timestamp):
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    start_of_day = dt.time(6, 0)
    current_day = days[timestamp.weekday() - 1]
    if timestamp.time() < start_of_day:
        return current_day
    else:
        return days[(timestamp.weekday()) % 7]

Final_analysis['CustomDayOfWeek'] = Final_analysis['APPT OPEN TS'].apply(custom_day_of_week)

In [39]:
# def custom_day_of_week(timestamp):
#     days = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
#     start_of_day = dt.time(6, 0)
#     current_day = days[timestamp.weekday()]
#     if timestamp.time() < start_of_day:
#         return days.index(current_day)
#     else:
#         return (days.index(current_day) + 1) % 7

# Final_analysis['CustomDayOfWeek'] = Final_analysis['APPT OPEN TS'].apply(custom_day_of_week)

In [ ]:
Final_analysis

In [ ]:
mismatch_df = Final_analysis.query('Plan_Run_Weekday != CustomDayOfWeek')
mismatch_df

In [42]:
# mismatch_df = mismatch_df['APPT OPEN WE DATE'] == mismatch_df['APPT OPEN WE DATE'].max
# mismatch_df

In [43]:
mismatch_df.to_csv('../Exports/Mismatches.csv', index=True)